# Introduction
This is a very rough first draft at importing and cleansing the data. Solution if heavily inspired by (okay... Completely ripped off) from https://gist.github.com/mbforbes/cee3fd5bb3a797b059524fe8c8ccdc2b


## Getting the content
Start by downloading the repository of (english) books. This is done in bash. Only tested on Ubuntu, but mac should work the same

```
wget -m -H -nd "http://www.gutenberg.org/robot/harvest?filetypes[]=txt&langs[]=en"
```
Takes a few hours to run, and is stored in a folder called rawContent. 
This is then copied to another folder, and we can start to clean up the mess

First we delete some dublications of the same books:
```
ls | grep "\-8.zip" | xargs rm
ls | grep "\-0.zip" | xargs rm
```
We can then unzip the files, and remove the zip files
```
unzip "*zip"
rm *.zip
```

Next we take care of some nested foldering
```
mv */*.txt ./
```
And finally, we remove all rubbish that isn't a real book:

```
ls | grep -v "\.txt" | xargs rm -rf
```


# Start the data cleansing

## Start with all imports at one place

In [1]:
from __future__ import absolute_import
from builtins import str
import os
from six import u

from os import listdir
from os.path import isfile, join

import nltk
import re
from operator import itemgetter    
import pandas as pd
from functools import reduce

## Next we define some constants
Much more are probably needed. only been testing on a few books at a time

In [2]:
file_path = "processedData"

TEXT_START_MARKERS = frozenset((u(_) for _ in (
    "*END*THE SMALL PRINT",
    "*** START OF THE PROJECT GUTENBERG",
    "*** START OF THIS PROJECT GUTENBERG",
    "This etext was prepared by",
    "E-text prepared by",
    "Produced by",
    "Distributed Proofreading Team",
    "Proofreading Team at http://www.pgdp.net",
    "http://gallica.bnf.fr)",
    "      http://archive.org/details/",
    "http://www.pgdp.net",
    "by The Internet Archive)",
    "by The Internet Archive/Canadian Libraries",
    "by The Internet Archive/American Libraries",
    "public domain material from the Internet Archive",
    "Internet Archive)",
    "Internet Archive/Canadian Libraries",
    "Internet Archive/American Libraries",
    "material from the Google Print project",
    "*END THE SMALL PRINT",
    "***START OF THE PROJECT GUTENBERG",
    "This etext was produced by",
    "*** START OF THE COPYRIGHTED",
    "The Project Gutenberg",
    "http://gutenberg.spiegel.de/ erreichbar.",
    "Project Runeberg publishes",
    "Beginning of this Project Gutenberg",
    "Project Gutenberg Online Distributed",
    "Gutenberg Online Distributed",
    "the Project Gutenberg Online Distributed",
    "Project Gutenberg TEI",
    "This eBook was prepared by",
    "http://gutenberg2000.de erreichbar.",
    "This Etext was prepared by",
    "This Project Gutenberg Etext was prepared by",
    "Gutenberg Distributed Proofreaders",
    "Project Gutenberg Distributed Proofreaders",
    "the Project Gutenberg Online Distributed Proofreading Team",
    "**The Project Gutenberg",
    "*SMALL PRINT!",
    "More information about this book is at the top of this file.",
    "tells you about restrictions in how the file may be used.",
    "l'authorization à les utilizer pour preparer ce texte.",
    "of the etext through OCR.",
    "*****These eBooks Were Prepared By Thousands of Volunteers!*****",
    "We need your donations more than ever!",
    " *** START OF THIS PROJECT GUTENBERG",
    "****     SMALL PRINT!",
    '["Small Print" V.',
    '      (http://www.ibiblio.org/gutenberg/',
    'and the Project Gutenberg Online Distributed Proofreading Team',
    'Mary Meehan, and the Project Gutenberg Online Distributed Proofreading',
    '                this Project Gutenberg edition.',
)))


TEXT_END_MARKERS = frozenset((u(_) for _ in (
    "*** END OF THE PROJECT GUTENBERG",
    "*** END OF THIS PROJECT GUTENBERG",
    "***END OF THE PROJECT GUTENBERG",
    "End of the Project Gutenberg",
    "End of The Project Gutenberg",
    "Ende dieses Project Gutenberg",
    "by Project Gutenberg",
    "End of Project Gutenberg",
    "End of this Project Gutenberg",
    "Ende dieses Projekt Gutenberg",
    "        ***END OF THE PROJECT GUTENBERG",
    "*** END OF THE COPYRIGHTED",
    "End of this is COPYRIGHTED",
    "Ende dieses Etextes ",
    "Ende dieses Project Gutenber",
    "Ende diese Project Gutenberg",
    "**This is a COPYRIGHTED Project Gutenberg Etext, Details Above**",
    "Fin de Project Gutenberg",
    "The Project Gutenberg Etext of ",
    "Ce document fut presente en lecture",
    "Ce document fut présenté en lecture",
    "More information about this book is at the top of this file.",
    "We need your donations more than ever!",
    "END OF PROJECT GUTENBERG",
    " End of the Project Gutenberg",
    " *** END OF THIS PROJECT GUTENBERG",
)))


LEGALESE_START_MARKERS = frozenset((u(_) for _ in (
    "<<THIS ELECTRONIC VERSION OF",
)))


LEGALESE_END_MARKERS = frozenset((u(_) for _ in (
    "SERVICE THAT CHARGES FOR DOWNLOAD",
)))

TITLE_MARKERS = frozenset((u(_) for _ in (
    "Title:",
)))

AUTHOR_MARKERS = frozenset((u(_) for _ in (
    "Author:",
)))
DATE_MARKERS = frozenset((u(_) for _ in (
    "Release Date:","Release Date:"
)))
LANGUAGE_MARKERS = frozenset((u(_) for _ in (
    "Language:",
)))
ENCODING_MARKERS = frozenset((u(_) for _ in (
    "Character set encoding:",
)))


# Define functions for preprocessing

## Read a single file

In [3]:
def read_file(file_name):
    file = open(file_name)
    file_content = file.read()

    lines = file_content.splitlines()
    sep = str(os.linesep)

    # Initialize results for single book
    content_lines = []
    i = 0
    footer_found = False
    ignore_section = False

    title = ""
    author = ""
    date = ""
    language = ""
    encoding = ""
    year = 0

    # Reset flags for each book
    title_found = False
    author_found = False
    date_found = False
    language_found = False
    encoding_found = False

    for line in lines:
            reset = False

            #print(line)
            if i <= 600:
                # Shamelessly stolen
                if any(line.startswith(token) for token in TEXT_START_MARKERS):
                    reset = True

                # Extract Metadata
                if title_found == False:
                    if any(line.startswith(token) for token in TITLE_MARKERS):
                        title_found = True
                        title = line
                if author_found == False:
                    if any(line.startswith(token) for token in AUTHOR_MARKERS):
                        author_found = True
                        author = line
                if date_found == False:
                    if any(line.startswith(token) for token in DATE_MARKERS):
                        date_found = True
                        date = line
                        year = int(re.findall(r'\d{4}', date)[0])
                if language_found == False:
                    if any(line.startswith(token) for token in LANGUAGE_MARKERS):
                        language_found = True
                        language = line
                if encoding_found == False:
                    if any(line.startswith(token) for token in ENCODING_MARKERS):
                        encoding_found = True
                        encoding = line

                # More theft from above
                if reset:
                    content_lines = []
                    continue

            # I feel like a criminal by now. Guess what? Also stolen
            if i >= 100:
                if any(line.startswith(token) for token in TEXT_END_MARKERS):
                    footer_found = True

                if footer_found:
                    break

            if any(line.startswith(token) for token in LEGALESE_START_MARKERS):
                ignore_section = True
                continue
            elif any(line.startswith(token) for token in LEGALESE_END_MARKERS):
                ignore_section = False
                continue

            if not ignore_section:
                if line != "": # Screw the blank lines
                    content_lines.append(line.rstrip(sep))
                i += 1

            sep.join(content_lines)

    # Do more cleaning
    for token in TITLE_MARKERS:
        title = title.replace(token, '').lstrip().rstrip()
    for token in AUTHOR_MARKERS:
        author = author.replace(token, '').lstrip().rstrip()
    for token in LANGUAGE_MARKERS:
        language = language.replace(token, '').lstrip().rstrip()
    for token in DATE_MARKERS:
        date = date.replace(token, '').lstrip().rstrip()
    for token in ENCODING_MARKERS:
        encoding = encoding.replace(token, '').lstrip().rstrip()
    return title, author, date, year, language, encoding, content_lines
    

## Return list of all words
Currently quite an empty function. However, I assume that some cleaning of cases etc. will be done here

In [4]:
def get_words(content_lines):
    all_text_lower = " ".join(content_lines).lower()
    words = re.findall(r'(\b[A-Za-z][a-z]{2,9}\b)', all_text_lower)

    # Do more cleansing. E.g. cases and stuff
    
    return words

## First attempt at actually creating statistics
Currently only a simple counting

In [5]:
def get_word_frequencies(words):
    frequency = {}
    for word in words:
        count = frequency.get(word,0)
        frequency[word] = count + 1

    word_count = len(words)
    unique_word_count = 0
    word_list = []
    word_list_count = []
    for key, value in reversed(sorted(frequency.items(), key = itemgetter(1))):
        word_list.append(key)
        word_list_count.append(value)
        unique_word_count = unique_word_count + 1
    
    word_list_freq = [freq / word_count for freq in word_list_count]
    
    word_freq = pd.DataFrame(list(zip(word_list, word_list_count, word_list_freq))
                             , columns = ['Word', 'count', 'freq'])
    
    word_freq['rank'] = word_freq['count'].rank(ascending = False, method="dense")

    return(word_freq, unique_word_count)

# Read all files, and do preprocessing
Well... Only five files currently

In [6]:
# Get all filenames
files = [f for f in listdir(file_path) if isfile(join(file_path, f))]

# Do only subset
files = files[0:20]

list_of_file = []
list_of_title = []
list_of_author = []
list_of_date = []
list_of_year = []
list_of_language = []
list_of_encoding = []
list_of_word_count = []
list_of_unique_word_count = []
list_of_word_frequencies = []

for file in files:
    # Read in basic information from file
    title, author, date, year, language, encoding, content_lines = read_file(file_path + "/" + file)
    line_count = len(content_lines)

    # Not sure if we want this for later:
    #content_all = " ".join(content_lines)
    
    # Split into words (and do various cleaning)
    words = get_words(content_lines)
    word_count = len(words)

    # First analysis, but should do something proper
    word_frequencies_table, unique_word_count = get_word_frequencies(words)

    
    # Append to results
    list_of_file.append(file)
    list_of_title.append(title)
    list_of_author.append(author)
    list_of_date.append(date)
    list_of_year.append(year)
    list_of_language.append(language)
    list_of_encoding.append(encoding)
    list_of_word_count.append(word_count)
    list_of_unique_word_count.append(unique_word_count)
    list_of_word_frequencies.append(word_frequencies_table)
    
    
    # Show basic information
    print("################################")
    print("################################")
    print("Filename: " + str(file))
    print("Title: " + str(title))
    print("Author(s): " + str(author))
    print("Date: " + str(date))
    print("Year: " + str(year))
    print("Language: " + str(language))
    print("Encoding: " + str(encoding))
    print("################################")
    print("Words in book: " + str(word_count))
    print("Unique words in book: " + str(unique_word_count))
    print("################################")
    print(word_frequencies_table)

# Feel free to change to dict? list? separate files?
## nested dataframes works, but looks super ungly when printing
### Fuck it - This is tooo useless killing it again
#all_res = pd.DataFrame(list(zip(list_of_file
#                                , list_of_title
#                                , list_of_author
#                                , list_of_date
#                                , list_of_language
#                                , list_of_encoding
#                                , list_of_word_count
#                                , list_of_unique_word_count
#                                , list_of_word_frequencies
#                                ))
#                             , columns = ['file'
#                                          , 'title'
#                                          , 'author'
#                                          , 'date'
#                                          , 'language'
#                                          , 'encoding'
#                                          , 'word_count'
#                                          , 'unique_word_count'
#                                          , 'word_frequencies'
#                                         ]
#                      )
                

################################
################################
Filename: 19586.txt
Title: The Simpkins Plot
Author(s): George A. Birmingham
Date: October 19, 2006  [eBook #19586]
Year: 2006
Language: English
Encoding: ISO-646-US (US-ASCII)
################################
Words in book: 60862
Unique words in book: 5135
################################
          Word  count      freq   rank
0          the   3905  0.064162    1.0
1          you   2175  0.035737    2.0
2          and   1809  0.029723    3.0
3         that   1585  0.026043    4.0
4         said   1157  0.019010    5.0
...        ...    ...       ...    ...
5130  includes      1  0.000016  175.0
5131      file      1  0.000016  175.0
5132   version      1  0.000016  175.0
5133      html      1  0.000016  175.0
5134   project      1  0.000016  175.0

[5135 rows x 4 columns]
################################
################################
Filename: 39628.txt
Title: The Expositor's Bible: The Pastoral Epistles
Author(s): A

################################
################################
Filename: 8859.txt
Title: True to the Old Flag
Author(s): G. A. Henty
Date: September, 2005
Year: 2005
Language: English
Encoding: ASCII
################################
Words in book: 85899
Unique words in book: 6046
################################
        Word  count      freq   rank
0        the   8864  0.103191    1.0
1        and   3563  0.041479    2.0
2        was   1505  0.017521    3.0
3       they   1305  0.015192    4.0
4       that   1158  0.013481    5.0
...      ...    ...       ...    ...
6041   raven      1  0.000012  187.0
6042  dragon      1  0.000012  187.0
6043   clive      1  0.000012  187.0
6044  author      1  0.000012  187.0
6045    tale      1  0.000012  187.0

[6046 rows x 4 columns]
################################
################################
Filename: 33906.txt
Title: The Sword of Deborah
Author(s): F. Tennyson Jesse
Date: October 25, 2010 [EBook #33906]
Year: 2010
Language: English
Enco

# Compare Word ranking between titles

In [7]:
list_count= []
list_freq = []
list_rank = []


col_names = list_of_title.copy()
col_names.insert(0,'Word')



for df in list_of_word_frequencies:
    list_count.append(df[['Word', 'count']])
    list_freq.append(df[['Word', 'freq']])
    list_rank.append(df[['Word', 'rank']])
    
df_count = reduce(lambda left, right: pd.merge(left, right, on="Word"), list_count)
df_count.columns = col_names
df_count['Sum'] = df_count.drop('Word', axis=1).apply(lambda x: x.sum(), axis=1)
df_count = df_count.sort_values(ascending = False, by=['Sum'])

df_freq = reduce(lambda left, right: pd.merge(left, right, on="Word"), list_freq)
df_freq.columns = col_names
df_freq['Avg'] = df_freq.drop('Word', axis=1).apply(lambda x: x.mean(), axis=1)
df_freq = df_freq.sort_values(ascending = False, by=['Avg'])

df_rank = reduce(lambda left, right: pd.merge(left, right, on="Word"), list_rank)
df_rank.columns = col_names
df_rank['Avg'] = df_rank.drop('Word', axis=1).apply(lambda x: x.mean(), axis=1)
df_rank = df_rank.sort_values(by=['Avg'])


In [8]:
#pd.set_option('display.max_rows', 500)
df_rank.head(30)

,Word,The Simpkins Plot,The Expositor's Bible: The Pastoral Epistles,A Lecture on the Study of History,Antwerp to Gallipoli,Heroines of Mormondom,Redemption and Two Other Plays,Dotty Dimple Out West,Casa Grande Ruin,"The Mirror of Literature, Amusement, and Instruction, No. 487",...,A Woman's Love,True to the Old Flag,The Sword of Deborah,"Encyclopaedia Britannica, 11th Edition, Volume 2, Slice 5",The Arrow of Fire,The Honour of Savelli,The Iron Boys in the Steel Mills,Defense of the Faith and the Saints (Volume 1 of 2),An Enquiry into the Causes of the Frequent Executions at Tyburn (1725),Avg
0,the,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00
1,and,3.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.15
2,that,4.0,3.0,3.0,5.0,10.0,4.0,9.0,3.0,3.0,...,4.0,5.0,3.0,6.0,4.0,6.0,5.0,3.0,3.0,5.00
4,was,6.0,11.0,19.0,3.0,3.0,28.0,3.0,4.0,7.0,...,3.0,3.0,5.0,3.0,3.0,3.0,4.0,9.0,29.0,7.65
5,for,8.0,6.0,11.0,10.0,7.0,11.0,12.0,21.0,9.0,...,8.0,11.0,4.0,9.0,11.0,7.0,7.0,5.0,5.0,8.65
7,with,12.0,12.0,17.0,4.0,11.0,14.0,11.0,10.0,5.0,...,13.0,8.0,7.0,8.0,12.0,5.0,12.0,11.0,8.0,9.75
11,not,16.0,5.0,12.0,15.0,17.0,10.0,14.0,13.0,10.0,...,8.0,18.0,11.0,13.0,13.0,11.0,8.0,6.0,13.0,12.15
9,but,14.0,9.0,21.0,18.0,13.0,6.0,12.0,11.0,12.0,...,10.0,12.0,10.0,11.0,18.0,9.0,20.0,16.0,13.0,12.95
10,his,15.0,7.0,15.0,14.0,9.0,19.0,18.0,44.0,4.0,...,6.0,10.0,65.0,4.0,5.0,4.0,6.0,7.0,10.0,14.25
6,have,11.0,13.0,27.0,30.0,31.0,7.0,16.0,31.0,13.0,...,18.0,15.0,16.0,19.0,24.0,14.0,14.0,10.0,9.0,17.65


In [9]:
df_freq.head(20)

,Word,The Simpkins Plot,The Expositor's Bible: The Pastoral Epistles,A Lecture on the Study of History,Antwerp to Gallipoli,Heroines of Mormondom,Redemption and Two Other Plays,Dotty Dimple Out West,Casa Grande Ruin,"The Mirror of Literature, Amusement, and Instruction, No. 487",...,A Woman's Love,True to the Old Flag,The Sword of Deborah,"Encyclopaedia Britannica, 11th Edition, Volume 2, Slice 5",The Arrow of Fire,The Honour of Savelli,The Iron Boys in the Steel Mills,Defense of the Faith and the Saints (Volume 1 of 2),An Enquiry into the Causes of the Frequent Executions at Tyburn (1725),Avg
0,the,0.064162,0.094727,0.063500,0.088397,0.079708,0.045150,0.055992,0.136344,0.112967,...,0.050256,0.103191,0.089332,0.124965,0.074572,0.079587,0.095594,0.115709,0.081075,0.087391
1,and,0.029723,0.045328,0.030223,0.051238,0.047835,0.034992,0.035048,0.035458,0.041803,...,0.047298,0.041479,0.042542,0.048356,0.024228,0.049154,0.021792,0.043850,0.053577,0.041816
2,that,0.026043,0.021651,0.012532,0.012070,0.010768,0.014818,0.012502,0.014880,0.013534,...,0.021378,0.013481,0.022284,0.009687,0.019016,0.012810,0.018030,0.026568,0.026395,0.016401
4,was,0.013752,0.008932,0.004528,0.013272,0.020509,0.005249,0.022012,0.013086,0.009277,...,0.024295,0.017521,0.011364,0.013671,0.020368,0.023981,0.018557,0.008963,0.003388,0.014113
5,for,0.011337,0.010683,0.006318,0.008099,0.011364,0.008141,0.009350,0.004221,0.007313,...,0.013502,0.008650,0.013983,0.008241,0.009156,0.011590,0.010984,0.009867,0.012370,0.009844
10,his,0.008232,0.010300,0.005581,0.005932,0.010877,0.006481,0.005236,0.001372,0.012988,...,0.014543,0.008673,0.001087,0.010051,0.015269,0.014844,0.013015,0.009357,0.009691,0.009224
7,with,0.009053,0.008472,0.004949,0.012149,0.009307,0.007302,0.010311,0.007915,0.009714,...,0.009710,0.009313,0.010327,0.008323,0.007888,0.013529,0.007799,0.007469,0.010400,0.009191
11,not,0.008067,0.012599,0.006213,0.005742,0.006494,0.008837,0.007907,0.007176,0.006876,...,0.013502,0.005099,0.008153,0.004903,0.007860,0.009625,0.010332,0.009813,0.008115,0.007970
9,but,0.008429,0.009096,0.004002,0.004793,0.007900,0.010462,0.009350,0.007387,0.005894,...,0.012918,0.007649,0.008400,0.006661,0.005916,0.010465,0.005417,0.005967,0.008115,0.007692
14,had,0.005767,0.004762,0.001422,0.011058,0.013203,0.002053,0.011594,0.000633,0.004912,...,0.013335,0.013097,0.007066,0.003649,0.011551,0.010587,0.009705,0.003918,0.002521,0.007397


In [10]:
df_count.head(20)

,Word,The Simpkins Plot,The Expositor's Bible: The Pastoral Epistles,A Lecture on the Study of History,Antwerp to Gallipoli,Heroines of Mormondom,Redemption and Two Other Plays,Dotty Dimple Out West,Casa Grande Ruin,"The Mirror of Literature, Amusement, and Instruction, No. 487",...,A Woman's Love,True to the Old Flag,The Sword of Deborah,"Encyclopaedia Britannica, 11th Edition, Volume 2, Slice 5",The Arrow of Fire,The Honour of Savelli,The Iron Boys in the Steel Mills,Defense of the Faith and the Saints (Volume 1 of 2),An Enquiry into the Causes of the Frequent Executions at Tyburn (1725),Sum
0,the,3905,8654,1206,5588,1473,2529,1048,1292,1035,...,1206,8864,1808,16847,2647,5871,3812,12935,1029,88910
1,and,1809,4141,574,3239,884,1960,656,336,383,...,1135,3563,861,6519,860,3626,869,4902,680,40973
2,that,1585,1978,238,763,199,830,234,141,124,...,513,1158,451,1306,675,945,719,2970,335,15804
4,was,837,816,86,839,379,294,412,124,85,...,583,1505,230,1843,723,1769,740,1002,43,13284
5,for,690,976,120,512,210,456,175,40,67,...,324,743,283,1111,325,855,438,1103,157,9313
10,his,501,941,106,375,201,363,98,13,119,...,349,745,22,1355,542,1095,519,1046,123,9133
7,with,551,774,94,768,172,409,193,75,89,...,233,800,209,1122,280,998,311,835,132,8733
11,not,491,1151,118,363,120,495,148,68,63,...,324,438,165,661,279,710,412,1097,103,7625
9,but,513,831,76,303,146,586,175,70,54,...,310,657,170,898,210,772,216,667,103,7281
14,had,351,435,27,699,244,115,217,6,45,...,320,1125,143,492,410,781,387,438,32,6795
